In [42]:
#r "..\Statsetera\bin\Debug\net6.0\Statsetera.dll"
#r "nuget:MathNet.Numerics"
// Install the ScottPlot NuGet package
#r "nuget:ScottPlot"
using MathNet.Numerics.Random;
using Statsetera;

// Setup a custom formatter to display plots as images
using Microsoft.DotNet.Interactive.Formatting;
Formatter.Register(typeof(ScottPlot.Plot), (plt, writer) => 
    writer.Write(((ScottPlot.Plot)plt).GetImageHTML()), HtmlFormatter.MimeType);

Installed Packages MathNet.Numerics, 5.0.0 ScottPlot, 4.1.60

Generate 100 observations from standard normal N(0, 1)

In [43]:
using MathNet.Numerics.Distributions;
RandomSource rng = new MersenneTwister();
int n = 100;
var samples = Normal.Samples(rng, 0, 1).Take(n);



Construct cdf F and compute 95% confidence band

In [44]:
var F = Stats.EmpiricalDistributionFunction(samples);
var alpha = 0.05;
var x = Enumerable.Range(-40000, 80000).Select(x => (double)x/10000);
var y = x.Select(x => (F(x, alpha), Normal.CDF(0, 1, x))).Aggregate(
    (0, 0), (acc, i) => 
    {
        var lower = i.Item1.Item2;
        var upper = i.Item1.Item3;
        var expected = i.Item2;
        return (lower <= expected && expected <= upper ? 
            acc.Item1 + 1 : acc.Item1, acc.Item2 + 1);
    }
);
(y, (double)y.Item1/(double)y.Item2)

Item1,Item2
"( 80000, 80000 )",1
